# data-driven approach

In [1]:
# !pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util
# Initialize the model
NLPmodel = SentenceTransformer('./sentence-transformers_all-MiniLM-L6-v2') # SentenceTransformer('all-MiniLM-L6-v2')

In [3]:
import pandas as pd
import numpy as np

In [4]:
# run if needed!!
# df with all verbs and their embeddings

# import nltk
# # nltk.download('wordnet')

# from nltk.corpus import wordnet as wn

# # Retrieve all English lemmas in WordNet
# lemmas = wn.all_lemma_names(pos=wn.VERB)#NOUN)

# # Convert lemmas to a list
# english_words = list(lemmas)

# print(english_words[:100]), len(english_words)

# english_words = [i for i in english_words if '-' not in i]
# english_words = [i for i in english_words if '_' not in i]
# english_words_embedding = NLPmodel.encode(english_words)

# print(english_words_embedding.shape)

# df_words = pd.DataFrame({'words':english_words, 'word_len':[len(i) for i in english_words], 'embedding':english_words_embedding.tolist()})
# df_words

In [5]:
# df_words.to_csv('../../data/text_responses/text_responses_all_verbs.csv')

In [6]:
df_words = pd.read_csv('../../data/text_responses/text_responses_all_verbs.csv',index_col=[0])
df_words.head()

,words,word_len,embedding
0,absent,6,"[0.019467102363705635, 0.05805996060371399, -0..."
1,abstract,8,"[-0.07030732184648514, 0.08017940074205399, 0...."
2,ace,3,"[-0.12659801542758942, 0.01382419653236866, -0..."
3,acuminate,9,"[-0.06811799108982086, 0.09903689473867416, -0..."
4,adulterate,10,"[0.03436809778213501, 0.04313870146870613, -0...."


In [7]:
english_words = df_words['words'].values
english_words_embedding = df_words['embedding'].values

In [8]:
param_name = 'chargeSpeed'
expt_type = 'playfight/with_cover_story/'
rootfile_loc = f'../../data/text_responses/{expt_type}'

In [9]:
df = pd.read_csv(f'{rootfile_loc}/data_triallevel_sorted.csv',index_col=[0])
df.rename(columns = {'chargeSpeeds':'chargeSpeed'},inplace=True)
df.head()

,subID,stimset_rows,chargeSpeed,responses,movie,trial_num
6,24093,10,1.5,They are peaceful,29e7c94f-a6bf-4f1d-8ea1-46efbdb6a836,6
6,24066,59,1.5,Slowly coming together to bump each other.,1a8cf3c1-690b-449c-bb36-fd3b251837c1,6
6,24067,90,1.5,Bumping into each other,e838dadb-c02f-4273-ae93-210720dcd06c,6
5,24068,57,1.5,They are kicking a ball.,31e1789e-459a-4f3d-bfdd-2f4c8c4e9f50,5
4,24069,42,1.5,playing frisbee,8ed7c0fc-1490-4e35-a000-7f7357a0772a,4


In [10]:
words_to_compare = df['responses'].to_list()
words_to_compare = [s.lower() for s in words_to_compare]
words_to_compare

['they are peaceful ',
 'slowly coming together to bump each other.',
 'bumping into each other',
 'they are kicking a ball.',
 'playing frisbee',
 'playing with each other',
 'the dots were slowly drifting together and apart in a repetitive way',
 'synchronised dancing',
 'playing together ',
 'interacting with each other by playing',
 'charging',
 'they were playing a game together',
 'playing independently near each other',
 'clashing',
 'running around',
 'slowly moving and touching one another',
 'chase',
 'playing a game',
 'catch with a ball',
 'playing',
 'running',
 'running around the park',
 'dancing',
 'playing red rover',
 'playing catch',
 'running toward each other',
 'just running around',
 'moving much slower around',
 'interacting or bumping into each other',
 'mirroring one another',
 'talking and exploring the park',
 'playing and handing each other objects',
 'running back and forth like a relay race',
 'the dots were gently bumping into each other repeatedly, much

In [11]:
excluded_words_list = ['dot','dots','grey','gray','black'] 

In [14]:
# word_embedding per chargeSpeed level - comparison to the whole verbs list
generative_responses =[]
# mean_embedding = []
for cspeed in np.unique(df[param_name]):
    print('chargeSpeed:', cspeed)
    df_cspeed = df.loc[df[param_name]==cspeed,:]
    words_to_compare = df_cspeed['responses'].to_list()
    words_to_compare =  [s.lower() for s in words_to_compare]
    mean_embedding_curr = NLPmodel.encode(words_to_compare)
    mean_embedding_curr = np.nanmedian(mean_embedding_curr,axis=0) 

    similarity_with_each_word = []
    for embedding in english_words_embedding: #english words
        similarity = util.pytorch_cos_sim(mean_embedding_curr, eval(embedding))
        similarity_with_each_word.append(similarity.mean().item())
    
    similarity_with_each_word = np.array(similarity_with_each_word)

    df_temp = pd.DataFrame({'english_words':english_words, 'similarity_with_each_word':similarity_with_each_word})
    df_temp = df_temp[~df_temp['english_words'].isin(excluded_words_list)]
    df_temp = df_temp.reset_index()
    df_temp.sort_values(by='similarity_with_each_word',inplace=True, ascending=False)
    print(f"most representative words: {df_temp.iloc[:5,1].values}")
    generative_responses.append(df_temp.iloc[:5,1].values)
    # take top 10 words

chargeSpeed: 1.5
most representative words: ['play' 'dance' 'game' 'sport' 'mingle']
chargeSpeed: 2.75
most representative words: ['play' 'game' 'outplay' 'sport' 'misplay']
chargeSpeed: 4.0
most representative words: ['play' 'game' 'sport' 'outplay' 'maneuver']
chargeSpeed: 5.25
most representative words: ['fight' 'combat' 'battle' 'play' 'game']
chargeSpeed: 6.5
most representative words: ['fight' 'sport' 'combat' 'battle' 'game']
chargeSpeed: 7.75
most representative words: ['play' 'fight' 'game' 'battle' 'combat']
chargeSpeed: 9.0
most representative words: ['fight' 'battle' 'fistfight' 'combat' 'play']


In [20]:
df_generative = pd.DataFrame({'chargeSpeed':np.unique(df[param_name]), 'most_representative_words':generative_responses[:]})
df_generative.sort_values(by='chargeSpeed',ascending=False,inplace=True)
df_generative

,chargeSpeed,most_representative_words
6,9.00,"[fight, battle, fistfight, combat, play]"
5,7.75,"[play, fight, game, battle, combat]"
4,6.50,"[fight, sport, combat, battle, game]"
3,5.25,"[fight, combat, battle, play, game]"
2,4.00,"[play, game, sport, outplay, maneuver]"
1,2.75,"[play, game, outplay, sport, misplay]"
0,1.50,"[play, dance, game, sport, mingle]"


In [21]:
def unique_words(curr_row):
    from collections import Counter
    
    all_rows = [0,1,2,3,4,5,6]
    all_other_rows = [x for x in all_rows if x != curr_row] # if curr_row is 0, this is [1,2,3,4]
    all_else = []
    for i in all_other_rows:
        all_else.extend(df_generative['most_representative_words'].values[i]) # if curr_row is 0, words from rows 1-4
    # all_else = np.array(all_else)
    # all_else = np.unique(all_else)
    word_count = Counter(all_else)
    # print(word_count)
    word_count = {word: count for word, count in word_count.items() if count == 6}
    all_else_6reps = word_count.keys()
    # print(all_else_6reps)
    curr_words = df_generative['most_representative_words'].values[curr_row]
    curr_words = [i for i in curr_words if i not in all_else_6reps]
    return curr_words

In [22]:
df_generative['most_representative_words'].values

array([array(['fight', 'battle', 'fistfight', 'combat', 'play'], dtype=object),
       array(['play', 'fight', 'game', 'battle', 'combat'], dtype=object),
       array(['fight', 'sport', 'combat', 'battle', 'game'], dtype=object),
       array(['fight', 'combat', 'battle', 'play', 'game'], dtype=object),
       array(['play', 'game', 'sport', 'outplay', 'maneuver'], dtype=object),
       array(['play', 'game', 'outplay', 'sport', 'misplay'], dtype=object),
       array(['play', 'dance', 'game', 'sport', 'mingle'], dtype=object)],
      dtype=object)

In [23]:
df_generative['most_representative_words'] = [unique_words(i) for i in range(7)]
df_generative

,chargeSpeed,most_representative_words
6,9.00,"[fight, battle, fistfight, combat, play]"
5,7.75,"[play, fight, game, battle, combat]"
4,6.50,"[fight, sport, combat, battle, game]"
3,5.25,"[fight, combat, battle, play, game]"
2,4.00,"[play, game, sport, outplay, maneuver]"
1,2.75,"[play, game, outplay, sport, misplay]"
0,1.50,"[play, dance, game, sport, mingle]"


In [24]:
results_loc = '../../results/text_analysis/pf/with_cover_story/'

In [25]:
df_generative.to_csv(f'{results_loc}/most_rep_words.csv',index=False)